# IIC #

In [1]:
print("Test")

Test


In [42]:
import numpy as np
import cv2

import random
import os

Functions used


In [41]:
def imshow(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def BoundaryFind(InIm):
    
    ret=InIm
    
    h = ret.shape[0]
    w = ret.shape[1]
    
    Max=0
    Min=255
    
    for y in range(0,h):
        for x in range(0, w):
            if(ret[y][x]>Max):
                Max=ret[y][x]
                #print("Max: ",Max)
            if(ret[y][x]<Min):
                Min=ret[y][x]
                #print("Min: ",Min)
    
    return (Max/2+Min/2)

def BoundaryCut(InIm):
    
    ret=InIm
    boundary=BoundaryFind(InIm)
    
    h = ret.shape[0]
    w = ret.shape[1]
    
    for y in range(0, h):
        for x in range(0, w):
          #print(ret[y][x])
          ret[y][x] = 255 if InIm[y][x] >= boundary else 0
    print("Boundary: ", boundary)
    
    return ret

In [53]:
def PreProcess(InIm):
    ZS=None
    ZS=InIm.copy()
    
    mean=cv2.mean(InIm)[0]
    var=np.var(InIm)
    
    print("Mean: ", mean)
    print("Var: ", var)
    
    
    h = ZS.shape[0]
    w = ZS.shape[1]
    
    
    
    kernel = np.ones((5,5),np.uint8)
    
    gradient = cv2.morphologyEx(ZS, cv2.MORPH_GRADIENT, kernel)
    #cv2.imshow('Gradient: ',gradient)
    
    closing = cv2.morphologyEx(gradient, cv2.MORPH_CLOSE, kernel,iterations= 2)
    #cv2.imshow('Closing: ',closing)
    
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel,iterations= 1)
    #cv2.imshow('Opening: ',opening)
    #ret = cv2.dilate(gradient,kernel,iterations = 3)
    
    ret=(255-opening)
    ret=BoundaryCut(ret)
    
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    
    print("Preprocess Done")
    return ret

Subimage class (we want to have each be a separate symbol, before determining the symbol)


In [131]:
boundary=10

class SubImage:
    def __init__(self,PointVector):
        n=PointVector.shape[0]
        
        print("n: ", n)
        
        xmin=np.Infinity
        xmax=0
        
        ymin=np.Infinity
        ymax=0
        
        
        for i in range(n):
            if(PointVector[i][1]<xmin):
                xmin=PointVector[i][1]
            if(PointVector[i][1]>xmax):
                xmax=PointVector[i][1]
            if(PointVector[i][0]<ymin):
                ymin=PointVector[i][0]
            if(PointVector[i][0]>ymax):
                ymax=PointVector[i][0]
        
        
        #print("Range: ( (",ymin,",",ymax,"),(",xmin,",",xmax,")")
        
        xmin=xmin-boundary
        xmax=xmax+boundary
        ymin=ymin-boundary
        ymax=ymax+boundary
        
        self.rot=0
        
        self.x=xmin
        self.y=ymin
        
        self.xsize=(int)(xmax-xmin)+1
        self.ysize=(int)(ymax-ymin)+1
        
        ZSim=np.zeros(shape=[self.ysize, self.xsize,1])#, dtype=np.uint8)
        
        #print("Size: (",self.ysize,",",self.xsize,")")
        
        for i in range(n):    
            #print("(",(PointVector[i][0]-ymin),",",(PointVector[i][1]-xmin),")" )
            ZSim[(int)(PointVector[i][0]-ymin+boundary/2)][(int)(PointVector[i][1]-xmin+boundary/2)][0]=255
        
        self.Image=ZSim
    Image=None
    x=0
    y=0
    xsize=0
    ysize=0
    rot=0

In [132]:
#just testing
n2=200
xsize2=1200
ysize2=1800
ymin=40
xmin=30

PointV=np.zeros(shape=[n2,2])

for i in range(n2):
    PointV[i][0]=random.randint(0,ysize2)+ymin
    PointV[i][1]=random.randint(0,xsize2)+xmin
    

#print(PointV)
SubIm1=SubImage(PointV)
#print(SubIm1.Image)
cv2.imshow('Subimage 1: ',SubIm1.Image)

cv2.waitKey(0)
cv2.destroyAllWindows()

n:  200


Subdividing Algorithm

In [133]:
def SubDividing(InIm):
    ZS=InIm.copy()
    SubDivList=[]
    
    h = InIm.shape[0]
    w = InIm.shape[1]
    
    mask = np.zeros((h + 2, w + 2), np.uint8)
    
    ZSVectorList=[]
    cv2.imshow('Before',ZS)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    for y in range(h):
        for x in range(w):
            if(ZS[y][x]==0):
                cv2.floodFill(ZS, mask, (x, y), 100);
                
                ZSVectorList=np.argwhere(ZS == 100)
                
                mask = np.zeros((h + 2, w + 2), np.uint8)
                cv2.floodFill(ZS, mask, (x, y), 255);
                
                print(ZSVectorList)
                
                ZSSubIm=SubImage(ZSVectorList)
                
                
                
                print(ZSSubIm)
                
                SubDivList.append(ZSSubIm)
                
                cv2.imshow('ZSSubIm: ',ZSSubIm.Image)
                
                
                cv2.imshow('ZS: ',ZS)
                
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                
                
                
    return SubDivList
    #source: https://www.programcreek.com/python/example/89425/cv2.floodFill
    
    

Process Analysing the Images (Basically step A)


In [134]:
def Process(InIm):
    
    cv2.imshow('Before',InIm)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    PPIm=PreProcess(InIm)
    SDIm=SubDividing(PPIm)
    
    return SDIm



Reading in images


In [135]:

Spam1 = cv2.imread('Spam1.jpg',0)
if(Spam1 is None):
    print("Spam1.jpg not found")

# Load an color image in grayscale
before1 = cv2.imread('Image1.png',0)
before2 = cv2.imread('Image2.png',0)
before3 = cv2.imread('Image3.png',0)


img = cv2.imread('SilverOwl.png',0)

if(before1 is None):
    print("before1.png not found")
if(before2 is None):
    print("before2.png not found")
if(before3 is None):
    print("before3.png not found")

Processing the images


In [136]:
im1=Process(before1)
#im2=Process(before2)
#im3=Process(before3)






Mean:  164.76252891159845
Var:  1026.7315023864421
Boundary:  168.0
Preprocess Done
[[ 278 1018]
 [ 278 1019]
 [ 278 1020]
 ...
 [ 508  934]
 [ 508  935]
 [ 508  936]]
n:  2133
[[282 863]
 [282 864]
 [282 865]
 ...
 [522 872]
 [522 873]
 [522 874]]
n:  2256
[[ 315 1185]
 [ 315 1186]
 [ 315 1187]
 ...
 [ 459 1186]
 [ 459 1187]
 [ 459 1188]]
n:  2380
[[319 144]
 [319 145]
 [319 146]
 ...
 [586 129]
 [586 130]
 [586 131]]
n:  2449
[[ 320 1054]
 [ 320 1055]
 [ 320 1056]
 ...
 [ 499 1070]
 [ 499 1071]
 [ 499 1072]]
n:  3717
[[333 827]
 [333 828]
 [333 829]
 ...
 [476 816]
 [476 817]
 [476 818]]
n:  2838
[[341 700]
 [341 701]
 [341 702]
 ...
 [472 705]
 [472 706]
 [472 707]]
n:  2672
[[364 194]
 [364 195]
 [364 196]
 ...
 [537 179]
 [537 180]
 [537 181]]
n:  4015
[[364 322]
 [364 323]
 [364 324]
 ...
 [514 355]
 [514 356]
 [514 357]]
n:  2451
[[375 443]
 [375 444]
 [375 445]
 ...
 [493 452]
 [493 453]
 [493 454]]
n:  2385
[[375 560]
 [375 561]
 [375 562]
 ...
 [476 568]
 [476 569]
 [476 570]

In [127]:
for i in range(len(im1)):
    name='SubImages\After'+str(i)+'.png'
    cv2.imwrite(name,im1[i].Image)

In [119]:

#SpamAfter1=Process(Spam1)

#cv2.imshow('Before Spam',Spam1)
#cv2.imshow('After Spam',SpamAfter1)

#cv2.waitKey(0)
#cv2.destroyAllWindows()

Showing the Images


In [36]:

cv2.imshow('Before 1',before1)
cv2.imshow('After 1',im1)

cv2.imshow('Before 2',before2)
cv2.imshow('After 2',im2)

cv2.imshow('Before 3',before3)
cv2.imshow('After 3',im3)

cv2.waitKey(0)
cv2.destroyAllWindows()

Saving the Images


In [13]:
cv2.imwrite('After1.png',im1)
cv2.imwrite('After2.png',im2)
cv2.imwrite('After3.png',im3)

True